In [2]:
import pandas as pd
import os
import numpy as np
import glob
import gdal
from gdalconst import GA_ReadOnly

In [2]:
idir = "R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\GRACE_GWS_Percentage_raw\\"
gws_list = glob.glob(idir+"gws*.img")
gws_list.sort()

In [8]:
idir = "R:\\UnitedStates\\ForDRI_project-NEW\\Case_Study_2022_Growing_Season\\GWS\\"
gws_list = glob.glob(idir+"gws*.tif")
gws_list.sort()

In [9]:
maskDs = gdal.Open('R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\INPUT_NEW_DATA_2022\\EDDI12\\EDDI200301.tif', GA_ReadOnly)# your mask raster
projection=maskDs.GetProjectionRef()
geoTransform = maskDs.GetGeoTransform()
minx = geoTransform[0]
maxy = geoTransform[3]
maxx = minx + geoTransform[1] * maskDs.RasterXSize
miny = maxy + geoTransform[5] * maskDs.RasterYSize

In [11]:
for f in gws_list:
    fname = os.path.basename(f).split('_')[4][:-4]
    data=gdal.Open(f, GA_ReadOnly) #Your data the one you want to clip
    output = 'R:\\UnitedStates\\ForDRI_project-NEW\\Case_Study_2022_Growing_Season\\GWS_raw\\GWS_'+fname+'.tif' #output file
    gdal.Translate(output,data,format='GTiff',projWin=[minx,maxy,maxx,miny],outputSRS=projection)
    print(output)

R:\UnitedStates\ForDRI_project-NEW\Case_Study_2022_Growing_Season\GWS_raw\GWS_20220103.tif
R:\UnitedStates\ForDRI_project-NEW\Case_Study_2022_Growing_Season\GWS_raw\GWS_20220110.tif
R:\UnitedStates\ForDRI_project-NEW\Case_Study_2022_Growing_Season\GWS_raw\GWS_20220117.tif
R:\UnitedStates\ForDRI_project-NEW\Case_Study_2022_Growing_Season\GWS_raw\GWS_20220124.tif
R:\UnitedStates\ForDRI_project-NEW\Case_Study_2022_Growing_Season\GWS_raw\GWS_20220131.tif
R:\UnitedStates\ForDRI_project-NEW\Case_Study_2022_Growing_Season\GWS_raw\GWS_20220207.tif
R:\UnitedStates\ForDRI_project-NEW\Case_Study_2022_Growing_Season\GWS_raw\GWS_20220214.tif
R:\UnitedStates\ForDRI_project-NEW\Case_Study_2022_Growing_Season\GWS_raw\GWS_20220221.tif
R:\UnitedStates\ForDRI_project-NEW\Case_Study_2022_Growing_Season\GWS_raw\GWS_20220228.tif
R:\UnitedStates\ForDRI_project-NEW\Case_Study_2022_Growing_Season\GWS_raw\GWS_20220307.tif
R:\UnitedStates\ForDRI_project-NEW\Case_Study_2022_Growing_Season\GWS_raw\GWS_20220314.tif

In [16]:
gws_list = glob.glob('R:\\UnitedStates\\ForDRI_project-NEW\\Case_Study_2022_Growing_Season\\GWS_raw\\' + '*.tif')
gws_list.sort()
mean_list = glob.glob('R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\Processing\\GRACE\\gws_mean\\' + '*.tif')
mean_list.sort()
std_list = glob.glob('R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\Processing\\GRACE\\gws_std\\' + '*.tif')
std_list.sort()

In [13]:
gws_arr = []
for gws in gws_list:
    ds = gdal.Open(gws)
    arr = ds.ReadAsArray()
    arr[arr==-999] = np.nan
    gws_arr.append(arr)
gws_arr = np.asarray(gws_arr)

In [ ]:
#if don't have mean and std images, run this
driver = gdal.GetDriverByName("GTiff")
for i in range(1,53):
    gws_month = gws_arr[i-1::52,:,:]
    gws_mean = np.mean(gws_month,0)
    mean = driver.Create(idir+'gws_mean\\MEAN_' + str(i)+'.tif', gws_mean.shape[1], gws_mean.shape[0], 1, gdal.GDT_Float32)
    mean.GetRasterBand(1).WriteArray(gws_mean)
    gws_std = np.std(gws_month,0)
    std = driver.Create(idir+'gws_std\\STD_' + str(i)+'.tif', gws_std.shape[1], gws_std.shape[0], 1, gdal.GDT_Float32)
    std.GetRasterBand(1).WriteArray(gws_std)
    print(i)
    mean.FlushCache()
    std.FlushCache()
    mean.SetGeoTransform(geoTransform)
    mean.SetProjection(projection)
    std.SetGeoTransform(geoTransform)
    std.SetProjection(projection)
    del gws_mean, gws_std

In [ ]:
driver = gdal.GetDriverByName("GTiff")
for i in range(52):
    if i+1 < 10:
        week = '0'+str(i+1)
    else:
        week = str(i+1)
    gws_mean = gdal.Open(mean_list[i])
    gws_mean_arr = gws_mean.ReadAsArray()
    gws_std = gdal.Open(std_list[i])
    gws_std_arr = gws_std.ReadAsArray()
    for j in range(18):
        gws = gdal.Open(gws_list[52*j+i])
        gws_arr = gws.ReadAsArray()
        projection = gws.GetProjectionRef()
        geoTransform = gws.GetGeoTransform()
        gws_zscore = (gws_arr-gws_mean_arr)/gws_std_arr
        year = str(2003+j)
        zscore = driver.Create(idir+'gws_zscore\\GWS_' + year+week+'.tif', gws_arr.shape[1], gws_arr.shape[0], 1, gdal.GDT_Float32)
        zscore.GetRasterBand(1).WriteArray(gws_zscore)
        zscore.SetGeoTransform(geoTransform)
        zscore.SetProjection(projection)
        zscore.FlushCache()
        print('GWS_' + year+week)
        del gws_arr

In [23]:
driver = gdal.GetDriverByName("GTiff")
for i in range(52):
    if i+1 < 10:
        week = '0'+str(i+1)
    else:
        week = str(i+1)
    gws_mean = gdal.Open(mean_list[i])
    gws_mean_arr = gws_mean.ReadAsArray()
    gws_std = gdal.Open(std_list[i])
    gws_std_arr = gws_std.ReadAsArray()

    gws = gdal.Open(gws_list[i])
    gws_arr = gws.ReadAsArray()
    projection = gws.GetProjectionRef()
    geoTransform = gws.GetGeoTransform()
    gws_zscore = (gws_arr-gws_mean_arr)/gws_std_arr
    year = '2022'
    zscore = driver.Create('R:\\UnitedStates\\ForDRI_project-NEW\\Case_Study_2022_Growing_Season\\GWS_zscore\\GWS_' + year+week+'.tif', gws_arr.shape[1], gws_arr.shape[0], 1, gdal.GDT_Float32)
    zscore.GetRasterBand(1).WriteArray(gws_zscore)
    zscore.SetGeoTransform(geoTransform)
    zscore.SetProjection(projection)
    zscore.FlushCache()
    print('GWS_' + year+week)
    del gws_arr

GWS_202201
GWS_202202
GWS_202203
GWS_202204
GWS_202205
GWS_202206
GWS_202207
GWS_202208
GWS_202209
GWS_202210
GWS_202211
GWS_202212
GWS_202213
GWS_202214
GWS_202215
GWS_202216
GWS_202217
GWS_202218
GWS_202219
GWS_202220
GWS_202221
GWS_202222
GWS_202223
GWS_202224
GWS_202225
GWS_202226
GWS_202227
GWS_202228
GWS_202229
GWS_202230
GWS_202231
GWS_202232
GWS_202233
GWS_202234


IndexError: list index out of range